# Question 11: 線形計画法 (7)

## 主婦問題: Google OR-Toolsでの解法
https://axross-recipe.com/recipes/97

=============================================================================================

問題）food.csv にリストアップされている７７種類の食料品から何をどのくらい購入すれば、nutrient.csvに記載された１日に健康維持に必要な栄養素を最安で充足することができるか？

Google OR-Tools を用いて求めよ。

このレシピでは、数理最適化を用いて1945年にG. Stiglerによって提案された「主婦問題」を解いてみます。
「主婦問題」は、スーパー・マーケットで売られている７７種の食料品の中から、何をどのくらいの量購入すれば、健康維持に必要な栄養素を最安で充足することができるか？という問題になります

主婦問題を提唱したG. Stigerですが、実はノーベル経済学賞を受賞しています。なので、彼の研究は経済政策に紐づくことが多く、主婦問題についても、食糧支給政策を決めるためのものなのです。 それでは、当時、どのような食材を組み合わせたレシピが最適だったのか、線形計画法を用いて算出していきましょう。なお、必要な栄養下限値や、食糧の価格・栄養素は当時の米国の数値になること、ご留意ください。

=============================================================================================

In [1]:
# Google driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# カレントディレクトリーを移動 --> 自分の Google driveでコードとデータを置いているフォルダを指定する
%cd /content/drive/MyDrive/公開講座/2022/04_Mathematical_optimization_1
%ls -lah

/content/drive/MyDrive/公開講座/2022/04_Mathematical_optimization_1
total 17M
-rw------- 1 root root  46K Feb  6 02:28  01_線型計画法_pulp実装.ipynb
-rw------- 1 root root  45K Feb  6 02:12  02_線形計画法_CVXPY実装.ipynb
-rw------- 1 root root  11K Feb  4 05:08  03_線形計画法_食事問題.ipynb
-rw------- 1 root root  53K Feb  6 05:10  04_混合整数計画法_ナップサック問題.ipynb
-rw------- 1 root root  79K Feb  5 11:54  05_線型計画法_主婦問題.ipynb
-rw------- 1 root root 937K Feb  5 16:13  06_混合整数計画法_倉庫配置_pulp_CBC実装.ipynb
-rw------- 1 root root 3.7M Feb  5 16:04  06_混合整数計画法_倉庫配置_pulp_CBC実装_local.ipynb
-rw------- 1 root root 2.1M Feb  6 13:21  06_混合整数計画法_倉庫配置_pulp_GLPK実装.ipynb
-rw------- 1 root root 930K Feb  5 11:55  06_混合整数計画法_倉庫配置_pulp実装.ipynb
-rw------- 1 root root 683K Feb  5 15:43  07_混合整数計画法_倉庫配置_CVXPY_CBC実装_local.ipynb
-rw------- 1 root root 1.8M Feb  6 12:17  07_混合整数計画法_倉庫配置_CVXPY_GLPK_MI実装.ipynb
-rw------- 1 root root 1.8M Feb  5 14:06  07_混合整数計画法_倉庫配置_CVXPY実装2.ipynb
-rw------- 1 root root 2.0M Feb  5 14:07  07_混合整数計画法_倉庫配置_CVXPY実装3

In [3]:
!pip install ortools

     |████████████████████████████████| 14.6 MB 16.2 MB/s 
     |████████████████████████████████| 1.1 MB 50.6 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3


In [4]:
from ortools.linear_solver import pywraplp
import time
import numpy as np
import pandas as pd

In [5]:
df_foods = pd.read_csv("./data/food.csv")
df_foods

,food,Calories,Protein,Calcium,Iron,Vitamin A,Thiamine (Vitamin B1),Riboflavin (Vitamin B2),Niacin,Ascorbic Acid (Vitamin C)
0,Wheat Flour (Enriched),44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,36.0,897,1.7,99,30.9,17.4,7.9,106,0
...,...,...,...,...,...,...,...,...,...,...
72,Chocolate,8.0,77,1.3,39,0.0,0.9,3.4,14,0
73,Sugar,34.9,0,0.0,0,0.0,0.0,0.0,0,0
74,Corn Syrup,14.7,0,0.5,74,0.0,0.0,0.0,5,0
75,Molasses,9.0,0,10.3,244,0.0,1.9,7.5,146,0


In [6]:
df_nutrient = pd.read_csv("./data/nutrient.csv")
df_nutrient

,Nutrient,Intake,Unit
0,Calories,3.0,kilocalories
1,Protein,70.0,grams
2,Calcium,0.8,grams
3,Iron,12.0,milligrams
4,Vitamin A,5.0,1000IU
5,Thiamine (Vitamin B1),1.8,milligrams
6,Riboflavin (Vitamin B2),2.7,milligrams
7,Niacin,18.0,milligrams
8,Ascorbic Acid (Vitamin C),75.0,milligrams


In [7]:
# ソルバーの定義
solver = pywraplp.Solver.CreateSolver('GLOP')

In [8]:
# 変数の定義
infinity = solver.infinity()
amounts= {}
for i in range(len(df_foods)):
  amounts[i] = solver.NumVar(0.0, infinity, "food_%i" % i)

In [9]:
# 以下実験コードをコメントアウト
# print(amounts)

In [10]:
# print(amounts.values())

In [11]:
# lst = []
# for i in range(len(df_nutrient)):
#   lst.append(np.dot(list(amounts.values()), list(df_foods.iloc[:,i+1])))

# print(lst)

In [12]:
# 制約条件
constraints = []
for i in range(len(df_nutrient)):
  constraints.append(solver.Add(np.dot(list(amounts.values()), list(df_foods.iloc[:,i+1])) >= df_nutrient.Intake[i]))

for j in range(len(df_foods)):
  constraints.append(solver.Add(amounts[i] >= 0))

constraints

[<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f4c00> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f0e10> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f0cf0> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f0bd0> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f0c00> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f0c90> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb0368f0d20> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <

In [13]:
# 目的関数の定義
# 最小化問題
solver.Minimize(solver.Sum([amounts[i] for i in range(len(amounts))]))

In [14]:
start = time.time()

# 解く
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

if result_status == pywraplp.Solver.OPTIMAL or result_status == pywraplp.Solver.FEASIBLE:
  print('Solution:')
  # The objective value of the solution.
  print('Optimal objective value = %f' % solver.Objective().Value())
else:
  print('The problem does not have an optimal solution.')

# 計算時間
print('Solving time:', time.time() - start, 's')

Solution:
Optimal objective value = 0.108662
Solving time: 0.004418611526489258 s


In [15]:
food_list = []
for i in range(len(amounts)):
  food_list.append(amounts[i].solution_value())
np.array(food_list)

array([0.02951906, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00189256,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01121444, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00500766, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06102856, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [16]:
# 年間コストを算出するため、最適値に365を乗じていっています。
annual_cost = 0
for i in range(len(amounts)):
    if amounts[i].solution_value() > 0:
        print("{0} : ${1}".format(df_foods.food[i], amounts[i].solution_value()))
        annual_cost += 365 * amounts[i].solution_value()
        
print("上記がコスト最小となる最適な食材の組み合わせで、年間{0}ドルで済む".format(annual_cost))

Wheat Flour (Enriched) : $0.029519061676488285
Liver (Beef) : $0.0018925572907052583
Cabbage : $0.01121443524614487
Spinach : $0.005007660466725204
Navy Beans, Dried : $0.061028563526693246
上記がコスト最小となる最適な食材の組み合わせで、年間39.66173154546625ドルで済む


はい、最適解は年間コスト$39.66ということになりました。物価が違うので参考にはなりませんが、小麦粉、牛レバー、キャベツ、ほうれん草、白インゲン豆...の5つの食材を使った料理が最もコスパの良い料理ということになりますね。

実は、Google社のシェフがこの食材を元に調理したことがあるとのことで、社員曰く「美味しかった」とのことです。...、興味ある方は是非お試しください！

https://ai.googleblog.com/2014/09/sudoku-linear-optimization-and-ten-cent.html